# Các thư viện cần sử dụng
- matplotlib: vẽ ảnh lên giao diện 
- numpy: hỗ trợ các phép toán với ma trận 
- sklearn: thư viện hỗ trợ các mô hình học máy - random forest, svm, các hàm hỗ trợ quá trình training cross-validate, ... 
- pyvi: thư viện hỗ trợ tách từ 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import learning_curve

from sklearn.datasets.base import load_files
from pyvi import ViTokenizer


from sklearn import svm
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.datasets.base import load_files
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

%matplotlib inline

## Load dữ liệu từ thư mục đã crappy từ trước 

Cấu trúc thư mục như sau 

- data_sample/news_1135/

    - Kinh tế: 
        - bài báo 1.txt 
        - bài báo 2.txt 
    - Pháp luật
        - bài báo 3.txt 
        - bài báo 4.txt 

In [2]:
data_train = load_files(container_path="data_sample/news_1135/", encoding="utf-8")

print(data_train.filenames)
print()

print("Tong so file: {}" .format( len(data_train.filenames)))




['data_sample/news_1135/Tin khác/0218e1df21ce358b9c6485176a48f1fcaeedef67.txt'
 'data_sample/news_1135/Khoa học - Công nghệ/bf9889f5f2ffd6c92fa877d35ef0ef5f34f0666d.txt'
 'data_sample/news_1135/Tin khác/d74aab054ffe9f8661df13bc52b438b48a63fe48.txt'
 ...
 'data_sample/news_1135/Thời sự/a06c1ec4c146d3b4eb5070a1967e10e5e21bdc5b.txt'
 'data_sample/news_1135/Sức khỏe/4187c4a1d528fd9ea4630d2709229df0b0d09c3d.txt'
 'data_sample/news_1135/Thể thao/7adaf0c561796f2411340150f18417543ad4403c.txt']

Tong so file: 1135


## Mô tả nội dung file 

- Nội dung file là text nội dung các bài báo đã qua module tách từ - sử dụng Vitokenizer thư viện pivy 
- Nhãn của văn bản là tên của folder mà bài báo đang nằm trong (Nhãn được chuyển hoá thành id dạng số)
- Danh sách 10 nhãn của tập dữ liệu ở dưới đây 
    - ['Giải trí', 'Khoa học - Công nghệ', 'Kinh tế', 'Pháp luật', 'Sức khỏe', 'Thể thao', 'Thời sự', 'Tin khác', 'Đời sống - Xã hội', 'Độc giả']

In [3]:
print(data_train.target_names)
print()

print(str(data_train.data[0]))
print(data_train.target[0])
print()

print(str(data_train.data[1]))
print(data_train.target[1])

['Giải trí', 'Khoa học - Công nghệ', 'Kinh tế', 'Pháp luật', 'Sức khỏe', 'Thể thao', 'Thời sự', 'Tin khác', 'Đời sống - Xã hội', 'Độc giả']

Dân_trí Sở GD & ĐT tỉnh Gia_Lai vừa ra văn_bản số 2258 / SGDĐT - VP , về việc chấn_chỉnh việc tiếp_thị sách và các vật_dụng khác trong các cơ_sở giáo_dục . Văn_bản chỉ_đạo , tuyệt_đối không cho phép các cá_nhân , tập_thể đến trường tiếp_thị , quảng_cáo mua_bán sách , dụng_cụ học_tập … cho giáo_viên và học_sinh trong nhà_trường . Các tổ_chức , cá_nhân trong ngành giáo_dục tuyệt_đối không được thực_hiện hoặc tham_gia giới_thiệu , quảng_bá , vận_động mua , phát_hành sách tham_khảo tới học_sinh hoặc phụ_huynh dưới hình_thức nào . Nhà_trường tuyệt_đối không được lưu_hành , sử_dụng sách có nội_dung không lành_mạnh , không phù_hợp với nội_dung chương_trình phổ_thông . Trường_hợp phát_hiện sách có sai_sót , các đơn_vị cần báo_cáo với cấp trên để có hướng xử_lý . Các sơ sở giáo_dục đề_cao cảnh_giác đối_với trường_hợp mạo_danh

## Thử nghiệm training dữ liệu với mô hình Random forest

- Dựng module tiền xử lý dữ liệu text, chuyển hoá text về dạng vector 
    - Sử dụng TfIdf vector để làm biểu diễn cho 1 văn bản 
    - Sử dụng training mô hình Doc2Vect - biểu diễn văn bản dưới dạng vector 
- Training dữ liệu với hàm fit. 
    - Dữ liệu input là vector văn bản / bài báo  
    - Dữ liệu nhãn là id của vector văn bản / bài báo 
- Save mô hình sau khi đã train. Và tái sử dụng cho việc infer sau này 

In [4]:
# load dữ liệu các stopwords 
with open("vietnamese-stopwords.txt") as f:
    stopwords = f.readlines()
stopwords = [x.strip().replace(" ", "_") for x in stopwords] 
print(stopwords[:10])

# 
# Transforming data 
# Chuyển hoá dữ liệu text về dạng vector tfidf 
#     - loại bỏ từ dừng
#     - sinh từ điển
module_count_vector = CountVectorizer(stop_words=stopwords)
model_rf_preprocess = Pipeline([('vect', module_count_vector),
                    ('tfidf', TfidfTransformer()),
                    ])
data_preprocessed = model_rf_preprocess.fit_transform(data_train.data, data_train.target)
print(module_count_vector.vocabulary_)


# Build model 
# Thử nghiệm với mô hình random forest classifier, với 50 cây quyết định (n_estimators), sử dụng hàm lỗi gini.  
model_rf =  RandomForestClassifier(criterion='gini', n_estimators=50)


# Training data 
# Gọi hàm fit với tham số 
#   - X: dữ liệu văn bản dạng vector 
#   - y: dữ liệu nhãn của văn bản (identify)
model_rf.fit(X=data_preprocessed, y=data_train.target)




['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy']
{'dân_trí': 6928, 'sở': 17869, 'gd': 7729, 'đt': 23214, 'tỉnh': 20851, 'gia_lai': 7816, 'văn_bản': 21779, '2258': 858, 'sgdđt': 17039, 'vp': 21572, 'chấn_chỉnh': 4971, 'tiếp_thị': 19345, 'sách': 17584, 'vật_dụng': 21910, 'cơ_sở': 5986, 'giáo_dục': 7955, 'chỉ_đạo': 5092, 'tuyệt_đối': 20254, 'phép': 15402, 'tập_thể': 20796, 'trường': 19928, 'quảng_cáo': 16194, 'mua_bán': 12653, 'dụng_cụ': 7191, 'học_tập': 9557, 'giáo_viên': 7961, 'học_sinh': 9552, 'nhà_trường': 14063, 'tổ_chức': 20928, 'ngành': 13667, 'tham_gia': 18129, 'giới_thiệu': 8119, 'quảng_bá': 16190, 'vận_động': 21905, 'mua': 12651, 'phát_hành': 15346, 'tham_khảo': 18130, 'phụ_huynh': 15666, 'hình_thức': 9214, 'lưu_hành': 11759, 'nội_dung': 14805, 'lành_mạnh': 11553, 'chương_trình': 4935, 'phổ_thông': 15653, 'trường_hợp': 19934, 'phát_hiện': 15342, 'sai_sót': 16816, 'báo_cáo': 3493, 'hướng': 9359, 'sơ': 17704, 'đề_cao': 23874, 'cảnh_giác': 6096, 'm

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## Thử nghiệm infer dữ liệu thực tế với mô hình Random forest sau khi đã training 

- Chuyển hoá dữ liệu text qua module tiền xử lý 
    - Do hệ thống cần truyền vào một chuỗi văn bản, nên với 1 văn bản cần infer thì ta truyền mảng có 1 phần tử 
- Dữ liệu thu được từ module tiền xử lý là vector đại diện của văn bản 
- Sử dụng mô hình đã được training để predict 

In [27]:
sentence = ViTokenizer.tokenize("""
HLV tuyển Việt Nam hài lòng với kết quả thắng 3-0 và ngôi đầu sau trận tiếp Campuchia tối 24/11.
HLV Felix Dalmas: 'Campuchia cố gắng, nhưng Việt Nam quá mạnh' / Việt Nam vào bán kết AFF Cup với ngôi đầu
"Hôm nay, chúng tôi thắng 3-0, các cầu thủ của tôi thi đấu tốt và tôi hài lòng. Nhưng nếu chỉ nhìn vào kết quả, sẽ rất dễ hiểu nhầm, bởi thực tế đây không hề là trận đấu dễ dàng", ông Park Hang-seo mở lời trong cuộc họp báo sau trận đấu.

Trên sân Hàng Đẫy tối 24/11, tuyển Việt Nam được đánh giá mạnh hơn hẳn so với Campuchia - đội đã bị loại sau ba lượt trận đầu. Tuy nhiên, trước lối đá phòng ngự dày đặc của đối phương, dàn cầu thủ dưới trướng HLV Park Hang-seo cũng phải chờ tới phút 39 mới có thể mở tỷ số, nhờ công tiền đạo Tiến Linh. Sau bàn khai thông, tuyển Việt Nam đá thoải mái hơn, nhưng cũng chỉ có thể ghi thêm hai bàn trong thời gian còn lại, với các pha lập công của Quang Hải ở phút 41, rồi Văn Đức ở phút 61.

Việt Nam 3-0 Campuchia    
Nhờ thắng Campuchia 3-0, cộng thêm việc Myanmar thua 0-3 trên sân Malaysia ở trận đấu cùng giờ, tuyển Việt Nam vượt lên chiếm ngôi nhất bảng A. Vị trị này được cho là sẽ mang lại lợi thế nhất định cho thầy trò HLV Park Hang-seo ở vòng knock-out, vì đội đứng thứ nhất ở vòng bảng sẽ đá trận bán kết lượt về trên sân nhà. Tuỳ kết quả các trận cuối bảng B ngày mai, tuyển Việt Nam sẽ biết Thái Lan, Philippines hay Singapore là đối thủ ở bán kết. Tuy nhiên, HLV Park Hang-seo không quá bận tâm về việc Việt Nam sẽ tranh vé vào chung kết với ai.

"Điều quan trọng nhất là tuyển Việt Nam cán đích ở vị trí dẫn đầu bảng. Ngay lúc này, chúng ta vẫn chưa biết sẽ đấu bán kết với đối thủ nào. Ngày mai, hai trợ lý của tôi sẽ chia nhau đi xem hai trận đấu ở Bangkok và Jakarta để tìm hiểu kỹ hơn về các đội có liên quan. Tuy nhiên, chúng tôi có nhiều video về cả ba đội có khả năng đi tiếp ở bảng B. Tuyển Việt Nam cũng có tới bảy ngày chuẩn bị cho trận bán kết lượt đi. Tôi tin mọi chuyện sẽ ổn", nhà cầm quân người Hàn Quốc cho biết.

HLV Park Hang-seo rất thoải mái khi nhắc đến vòng bán kết. 
HLV Park Hang-seo rất thoải mái khi nhắc đến vòng bán kết. 

Đang thoải mái và vui vẻ khi nói về trận đấu và vòng bán kết, HLV Park Hang-seo chợt chùng giọng xuống khi được hỏi về học trò Nguyễn Văn Toàn. Cầu thủ thuộc biên chế HAGL dính chấn thương trong buổi tập hôm qua 23/11, và không có tên trong danh sách đăng ký đấu Campuchia hôm nay. Khi xuất hiện trên sân Hàng Đẫy tối 24/11, Văn Toàn cũng phải nhờ các đồng đội đỡ, dìu hoặc cõng đi.  

"Văn Toàn có phần suy sụp vì chấn thương hôm qua. Theo tính toán ban đầu của tôi, lẽ ra cậu ấy sẽ đá chính trước Campuchia hôm nay. Qua đánh giá ban đầu, tôi nghĩ chấn thương của Văn Toàn không quá nghiêm trọng. Nhưng chúng ta vẫn phải chờ kết luận bác sỹ. Đến giờ, tôi vẫn chưa biết liệu cậu ấy có thể trở lại hay không. Tôi chỉ hy vọng Văn Toàn sẽ bình phục càng sớm càng tốt", HLV trưởng tuyển Việt Nam nói. 


""")
sentence_vector = model_rf_preprocess.transform([sentence])
print(sentence)
print(sentence_vector[0] )

out_label = model_rf.predict(sentence_vector)
print(data_train.target_names)
print(out_label[0])

data_train.target_names[out_label[0]]

print(data_preprocessed[0][:30])
data_train.target_names[out_label[0]]




HLV tuyển Việt_Nam hài_lòng với kết_quả thắng 3-0 và ngôi đầu sau trận tiếp Campuchia tối 24/11 . HLV Felix_Dalmas : ' Campuchia cố_gắng , nhưng Việt_Nam quá mạnh ' / Việt_Nam vào bán_kết AFF Cup với ngôi đầu " Hôm_nay , chúng_tôi thắng 3-0 , các cầu_thủ của tôi thi_đấu tốt và tôi hài_lòng . Nhưng nếu chỉ nhìn vào kết_quả , sẽ rất dễ hiểu nhầm , bởi thực_tế đây không hề là trận_đấu dễ_dàng " , ông Park_Hang - seo mở lời trong cuộc họp_báo sau trận_đấu . Trên sân Hàng Đẫy tối 24/11 , tuyển Việt_Nam được đánh_giá mạnh hơn hẳn so với Campuchia - đội đã bị loại sau ba lượt trận đầu . Tuy_nhiên , trước lối đá phòng_ngự dày_đặc của đối_phương , dàn cầu_thủ dưới trướng HLV Park_Hang - seo cũng phải chờ tới phút 39 mới có_thể mở tỷ_số , nhờ công tiền_đạo Tiến_Linh . Sau bàn khai_thông , tuyển Việt Nam_đá thoải_mái hơn , nhưng cũng chỉ có_thể ghi thêm hai bàn trong thời_gian còn lại , với các pha lập_công của Quang_Hải ở phút 41 , rồi Văn_Đức ở phút 61 . Việt_Nam 3-0 Campuchia Nhờ thắng Campuch

'Thể thao'